In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import random
from ortools.sat.python import cp_model
import time
import locust

In [2]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
#start_time = time.time()

# users-melbcbd-generated.csv contains:
# •  Latitude-Longitude
# of the users in the Melbourne CBD area.
requests_path = '..\\eua-dataset\\users\\'
R = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users
#R = pd.read_csv(requests_path + 'users-test-more-users.csv') # test with 14 users
#R = pd.read_csv(requests_path + 'users-melbcbd-generated.csv') # test with 816 users

# site-optus-melbCBD.csv contains:
# •  SiteID-Latitude-Longitude-Name-State-LicensingAreaID-PostCode-SitePrecision-Elevation-HCISL2
# of all Optus BS in Melbourne CBD area (edge-servers)
nodes_path = '..\\eua-dataset\\edge-servers\\'
#N = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_src = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
N_dest = pd.read_csv(nodes_path + 'serverstest.csv') # test with 3 servers
#N = pd.read_csv(nodes_path + 'serverstest-more-servers.csv') # test with 6 servers
#N = pd.read_csv(nodes_path + 'site-optus-melbCBD.csv') # test with 125 servers

In [3]:
#-----------------------------------INPUTS-----------------------------------
# Memory of function: m_f
m1 = 1
m2 = 2
m3 = 3
m4=4
m_f = [m1,m2,m3,m4]

#function assigned to each request
m_request = []
""" for j in range(len(R)):
  m = random.choice([m1, m2, m3,m4])
  m_request.append(m) """
m_request= [m1,m2,m3,m4,m1,m2,m3,m4]
#m_request= [m1,m2,m3,m4,m1,m2,m3,m4,m1,m2,m3,m4,m1,m2]
index = sorted(range(len(m_request)), key=lambda a: m_request[a]) # Sort the requests by their requirements

#shows which requests are assigned to each function
req_distr=np.zeros([len(m_f),len(R)])
for m in range(len(m_f)):
  for r in range(len(R)):
    if m_request[r]==m_f[m]:
      req_distr[m][r]=1
    else:
      req_distr[m][r]=0  

#Maximum allowed network delay for function f:  phi_f
phi_f=[]


In [4]:
#-----------------------------------INFRASTRUCTURED DATA-----------------------------------

#Memory availably in node j: M_j
M_j = [20,21,22]

#Cores on node j: U_j
U_j=[25,26,27]

In [5]:
#-----------------------------------MONITORED DATA-----------------------------------
#Network delay between nodes i and j
delta_ij = []

#Incoming requests to node i
lambda_ri = [[1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1]]
#lambda_fi =[[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2]]

#Cores used by node j per single request: u_rj
u_rj = [[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2],[2,2,2,2,2,2,2,2]]

In [6]:
#-----------------------------------VARIABLES-----------------------------------

U_ni = [] # Set of requests conected to node n

#establish which containers (function instances) are allocated to each node (????????????)
container_loc = [[1,0,1,0],[0,1,0,1],[1,0,0,1]]

#Groups requests by functions
F_r=[]
for row in range(len(req_distr)):
    temp_F=[]
    for column in range(len(R)):
        if req_distr[row][column]==1:
            temp_F.append(column)
    F_r.append(temp_F)

In [7]:
#-----------------------------------HAVERSINE-----------------------------------

def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [8]:
#-----------------------------------COVERAGE REQUEST-NODE-----------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node_latitude = N_src.iloc[i]['LATITUDE']
  node_longitude = N_src.iloc[i]['LONGITUDE']
  temp = []
  for r in range(len(R)):
    request_latitude = R.iloc[r]['Latitude']
    request_longitude = R.iloc[r]['Longitude']

    dist_geo = haversine(node_longitude, node_latitude, request_longitude, request_latitude)
    
    if dist_geo <= radius[i]:
        temp.append(1)
        
    else:
        temp.append(0)
       
  U_ni.append(temp)


In [9]:
#---------------------------------DISTANCE BETWEEN NODES---------------------------------

#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(N_src), 0.03)

for i in range(len(N_src)):
  node1_latitude = N_src.iloc[i]['LATITUDE']
  node1_longitude = N_src.iloc[i]['LONGITUDE']
  temp_dist = []
  for j in range(len(N_dest)):
    node2_latitude = N_dest.iloc[j]['LATITUDE']
    node2_longitude = N_dest.iloc[j]['LONGITUDE']

    dist_geo_nodes = haversine(node1_longitude, node1_latitude, node2_longitude,node2_latitude)
    #print(dist_geo)
    #print(radius[i]+radius[j])
    temp_dist.append(np.round(dist_geo_nodes,3))
       
  delta_ij.append(temp_dist)

for r1 in range(len(N_src)):
  temp_delay = []
  for r2 in range(len(N_dest)):
    if r1==r2:
      temp_delay.append(0)
    else:
      temp_delay.append(radius[r1]+radius[r2])
  phi_f.append(temp_delay)
    

In [10]:
#---------------------------------- CP MODEL ----------------------------------
model = cp_model.CpModel()

#-----------------------------------SOLVER VARIABLES-----------------------------------

# x_j,r = True if request r is allocated to node j
x = {}
for r in index:
    for j in range(len(N_dest)):
        x[j, r] = model.NewBoolVar(f'c[{j}][{r}]')

# c_f,j = True if container f (fa,fb,fc) is deployed on node j
c = {}
for f in range(len(F_r)):
    for j in range(len(N_dest)):
        c[f, j] = model.NewBoolVar(f'c[{f}][{j}]')

# y_j = True if node j is used
# y_j = False otherwise
y = {}
for j in range(len(N_dest)):
    y[j] = model.NewBoolVar(f'c[{j}]')

print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]



In [11]:
#-----------------------------------CONSTRAINTS-----------------------------------

# Proximity constraint (request-node) *******OJOOOOOO*******
for r in index:
    for i in range(len(N_src)):
        if U_ni[i][r] == 0:
            model.Add(
             x[j, f]==0 #no manejamos j loop
            )

#Proximity constraint (node i-node j)
for i in range(len(N_src)):
    for j in range(len(N_dest)):
        if delta_ij[i][j]> phi_f[i][j]:
            model.Add(
             x[j, r]==0
            )

#Container deployment: True if container of function f is deployed on node j (??????????????? x se maneja con i o j)
""" for r in index:
    for f in range(len(F_r)):
        for j in range(len(N_dest)):
            model.Add(sum([x[i,r] for i in range(len(N_src))])<= c[f,j]) """


# Capacity constraint (memory)
for j in range(len(N_dest)):
        model.Add(
            sum([
                m_f[f] * c[f,j] for f in range(len(F_r))
            ]) <= M_j[j]*y[j]
        )

# Avoid resource contention
for j in range(len(N_dest)):
    model.Add(
            sum([
                sum([x[j,r]*lambda_ri[i][r]*u_rj[j][r] for r in index]) 
            for i in range(len(N_src))
            ]) <= U_j[j]
        )  


# Contraint family (each user can be allocated just once)
for r in index:
    model.Add(
        sum([
            x[j, r] for j in range(len(N_dest))
        ]) <= 1
    ) 

print(model.ModelStats())

satisfaction model '':
#Variables: 39
  - 39 Booleans in [0,1]
#kLinear1: 11
#kLinear3: 8
#kLinearN: 6 (#terms: 39)


In [12]:
# Declare our intermediate boolean variable.
b = model.NewBoolVar('b')

for r in index:
    for f in range(len(F_r)):
        
        model.Add(sum([x[j,r] for j in range(len(N_dest))])>0).OnlyEnforceIf(b)
        model.Add(sum([x[j,r] for j in range(len(N_dest))])==0).OnlyEnforceIf(b.Not())

        model.Add(c[f,j]==1).OnlyEnforceIf(b)
        model.Add(c[f,j]==0).OnlyEnforceIf(b.Not())

In [13]:
#---------------------------------- CP SOLVER ----------------------------------
solver = cp_model.CpSolver()

#---------------------------------- OBJECTIVE FUNCTIONS ---------------------------------

# Maximize the number of allocated requests
objective_max = []
for r in index:
    for j in range(len(N_dest)):
        objective_max.append(x[j, r])
model.Maximize(sum(objective_max))

solver.Solve(model)
max_requests = solver.ObjectiveValue()

# Hint (speed up solving)
for r in index:
    for j in range(len(N_dest)):
        model.AddHint(x[j,r], solver.Value(x[j,r]))

for j in range(len(N_dest)):
    model.AddHint(y[j], solver.Value(y[j]))
        
# Constraint previous objective
model.Add(
    sum([
        x[j, r] for j in range(len(N_dest)) for r in index
    ]) == round(solver.ObjectiveValue())
)

# Minimize the number of nodes used
objective_min = []
for j in range(len(N_dest)):
    objective_min.append(y[j])
model.Minimize(sum(objective_min))

print(model.ModelStats())

optimization model '':
#Variables: 40 (#bools:3 in objective)
  - 40 Booleans in [0,1]
#kLinear1: 75 (#enforced: 64)
#kLinear3: 72 (#enforced: 64)
#kLinearN: 7 (#terms: 63)


In [14]:
#-----------------------------------CALL THE SOLVER-----------------------------------
status = solver.Solve(model)

#-----------------------------------DISPLAY THE SOLUTION-----------------------------------

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    print(f'Objective value: {max_requests} requests have been allocated to {solver.ObjectiveValue()} nodes\n')
    for r in index:
        for j in range(len(N_dest)):
            if int(solver.Value(x[j,r])) == 1:
                print(f'Request {r} has been allocated on node {j}')
else:
    print('The problem does not have an optimal solution.')

#print("\n--- Run time: %s seconds ---" % round((time.time() - start_time),2))

Solution:
Objective value: 8.0 requests have been allocated to 1.0 nodes

Request 0 has been allocated on node 0
Request 4 has been allocated on node 1
Request 1 has been allocated on node 1
Request 5 has been allocated on node 2
Request 2 has been allocated on node 0
Request 6 has been allocated on node 2
Request 3 has been allocated on node 1
Request 7 has been allocated on node 2


In [15]:
from locust import HttpUser, task

class HelloWorldUser(HttpUser):
    @task
    def hello_world(self):
        self.client.get("/hello")
        self.client.get("/world")

